In [1]:
import pandas as pd
import glob
import numpy as np
import re
from tqdm.auto import tqdm
import pickle
from pandarallel import pandarallel
import re
import plotly.graph_objects as go
import time
import requests

pandarallel.initialize()

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [ ]:
df = pd.read_parquet("../data/comp/categories.parquet.gzip")

In [3]:
df

,state,district,block,category
0,andaman & nicobar,n & m andaman,porlob island,safe
1,andaman & nicobar,n & m andaman,north passage island,safe
2,andaman & nicobar,n & m andaman,interview island,safe
3,andaman & nicobar,n & m andaman,stewart island,safe
4,andaman & nicobar,n & m andaman,narcondam island,safe
...,...,...,...,...
6757,west bengal,uttar dinajpur,karandighi,safe
6758,west bengal,uttar dinajpur,kaliaganj,safe
6759,west bengal,uttar dinajpur,goalpokhar -ii,safe
6760,west bengal,uttar dinajpur,raiganj,safe


In [4]:
# Changed all state in category file to name found in gw file

### Check for same values

In [5]:
df_pre = pd.read_parquet("../data/comp/gw-block-pre-post.parquet.gzip")

In [6]:
df_pre.shape

(5369, 86)

In [7]:
def changed_name(type_v):
    v = list(set(df.loc[:, type_v].values))
    not_found = []
    for i in list(set(df_pre.loc[:, type_v].values)):
        found = False
        if i in v:
            found = True
        if not found:
            not_found.append(i)
    return not_found

In [8]:
state = changed_name("state")
district = changed_name("district")
block = changed_name("block")

In [9]:
state

[]

In [ ]:
df = df.replace(
    {
        "andaman & nicobar is": "andaman & nicobar",
        "jammu&kashmir": "jammu & kashmir",
        "tamil nadu": "tamilnadu",
        "jharkhund": "jharkhand",
        "daman&diu": "daman & diu",
        "dadra & nagar haveli": "dadra and nagar hav",
    }
)

In [10]:
len(district)

109

In [11]:
len(block)

1802

In [12]:
df_categories = pd.read_csv("../data/blocks_not_found_categories.csv")

### Call api

#### blocks

In [13]:
%time
found_list = []
not_found_list = []
for i in tqdm(df_categories.iterrows()):
    state_name = i[1]["state"].strip()
    state = state_name.replace(" ", "+")
    block_name = i[1]["block"].strip()
    block = block_name.replace(" ", "+")
    url = (
        "https://nominatim.openstreetmap.org/search.php?street=&city=&county="
        + block
        + "&state="
        + state
        + "&country=india&postalcode=&polygon_geojson=1&format=geojson"
    )
    r = requests.get(url, allow_redirects=True)
    if r.status_code != 200:
        break
    else:
        res = json.loads(r.text)
        found = len(res["features"])
        if found >= 1:
            found_list.append(block_name + "-_-" + state_name)
            with open(
                r"../data/geojson-cat-blocks/" + block_name + ".json", "w"
            ) as f:
                json.dump(res, f)
        else:
            not_found_list.append(block_name + "-_-" + state_name)
    time.sleep(1)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.25 µs


KeyboardInterrupt: 

#### districts

In [5]:
df = df.drop_duplicates(subset=["district"]).reset_index(drop=True)

In [6]:
temp = pd.read_csv("../data/districts_not_found_categories.csv")

In [7]:
df = temp

In [8]:
%time
found_list = []
not_found_list = []
for i in tqdm(df.iterrows()):
    state_name = i[1]["state"].strip()
    state = state_name.replace(" ", "+")
    district_name = i[1]["district"].strip()
    district = district_name.replace("_", " ")
    district = district.replace(" ", "+")
#     print(district)
    url = (
        "https://nominatim.openstreetmap.org/search.php?street=&county="
        + district
        + "&state=&country=india&postalcode=&polygon_geojson=1&format=geojson"
    )
    r = requests.get(url, allow_redirects=True)
    if r.status_code != 200:
        break
    else:
        res = json.loads(r.text)
        found = len(res["features"])
        if found >= 1:
            found_list.append(district_name + "-_-" + state_name)
            with open(
                r"../data/geojson-cat-district/" + district_name + ".json", "w"
            ) as f:
                json.dump(res, f)
        else:
            not_found_list.append(district_name + "-_-" + state_name)
    time.sleep(1)
#     print(url)

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 7.87 µs


arvalli
banaskantha
bhabhua
bhadradri+kothagudem
bolangir
bulandshahar
chamrajnagara
charki+dadri
chhota+udepur
chikballapur
dakshin+kannada
dantewara
davangere
jayashankar+bhupalapally
jogulamba+gadwal
kamrup+metro
kasargod
komarambhem+asifabad
kushi+nagar
lakhimpur+kheri
medchal+malkajgiri
morigaon
nawanshahar
non+revenue+unit
palamau
panchmahal
peramabalur
raibareli
rajanna+sircilla
sabarkantha
saraikela+-+kharsawan
sheihkpura
thiruvannamalai
tiruvarur
tuenchung
unakoti
uttar+kannada
villupuram
viruthunagar
warangal+rural
warangal+urban
yadadri+bhongiri



In [9]:
len(not_found_list)

35

In [10]:
not_found_list

['arvalli-_-gujarat',
 'banaskantha-_-gujarat',
 'bhabhua-_-bihar',
 'bolangir-_-odisha',
 'bulandshahar-_-uttar pradesh',
 'chamrajnagara-_-karnataka',
 'charki dadri-_-haryana',
 'chhota udepur-_-gujarat',
 'chikballapur-_-karnataka',
 'dakshin kannada-_-karnataka',
 'dantewara-_-chhattisgarh',
 'davangere-_-karnataka',
 'kamrup metro-_-assam',
 'kasargod-_-kerala',
 'komarambhem_asifabad-_-telangana',
 'kushi nagar-_-uttar pradesh',
 'lakhimpur kheri-_-uttar pradesh',
 'morigaon-_-assam',
 'nawanshahar-_-punjab',
 'non revenue unit-_-delhi',
 'palamau-_-jharkhand',
 'panchmahal-_-gujarat',
 'peramabalur-_-tamilnadu',
 'raibareli-_-uttar pradesh',
 'sabarkantha-_-gujarat',
 'saraikela - kharsawan-_-jharkhand',
 'sheihkpura-_-bihar',
 'thiruvannamalai-_-tamilnadu',
 'tiruvarur-_-tamilnadu',
 'tuenchung-_-nagaland',
 'unakoti-_-tripura',
 'uttar kannada-_-karnataka',
 'villupuram-_-tamilnadu',
 'viruthunagar-_-tamilnadu',
 'yadadri_bhongiri-_-telangana']

In [11]:
not_found_district = []
not_found_state=[]
for i in not_found_list:
    not_found_district.append(i.split("-_-")[0])
    not_found_state.append(i.split("-_-")[1])

In [12]:
temp = pd.DataFrame({"district": not_found_district,"state":not_found_state})

In [13]:
temp.to_csv("../data/districts_not_found_categories.csv", index=False)

In [27]:
df.loc[:, "state"].unique()

array(['andaman & nicobar', 'andhra pradesh', 'arunachal pradesh',
       'assam', 'bihar', 'chandigarh', 'chhattisgarh',
       'dadra and nagar hav', 'daman & diu', 'delhi', 'goa', 'gujarat',
       'haryana', 'himachal pradesh', 'jammu & kashmir', 'jharkhand',
       'karnataka', 'kerala', 'lakshadweep', 'madhya pradesh',
       'maharashtra', 'manipur', 'meghalaya', 'mizoram', 'nagaland',
       'odisha', 'pondicherry', 'punjab', 'rajasthan', 'sikkim',
       'tamilnadu', 'telangana', 'tripura', 'uttar pradesh',
       'uttarakhand', 'west bengal'], dtype=object)

In [29]:
df.to_parquet("../data/comp/categories.parquet.gzip", compression="gzip")
df.to_csv("../data/csv/categories.csv")

In [ ]:
# were errors now fixed
# "jharkhand",
# "tamilnadu",
# "andaman & nicobar",
# "jammu & kashmir",
# "daman & diu",
# "dadra and nagar hav"